<a href="https://colab.research.google.com/github/Gk1694/Natural-language-processing/blob/main/Chatbot_for_doc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
mkdir data

In [ ]:
!pip install langchain
!pip install openai

In [ ]:
!pip install chromadb tiktoken docx2txt

In [ ]:
!pip install pypdf

In [5]:
from chromadb.api.types import Metadatas
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

os.environ["OPENAI_API_KEY"] = "<YOUR API KEY>"

# load the document 
loader = PyPDFLoader('ssh275.pdf')
documents = loader.load()

# we split the data into chunks of 1,000 characters, with an overlap
# of 200 characters between the chunks, which helps to give better results
# and contain the context of the information between chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(documents)

# we create our vectorDB, using the OpenAIEmbeddings tranformer to create
# embeddings from our text chunks. We set all the db information to be stored
# inside the ./data directory, so it doesn't clutter up our source files
vectordb = Chroma.from_documents(
  documents,
  embedding=OpenAIEmbeddings(),
  persist_directory='./data'
)
vectordb.persist()

In [6]:
print(len(documents))

12


In [19]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

def answer(query):
  qa_chain = RetrievalQA.from_chain_type(
    llm=OpenAI(),
    retriever=vectordb.as_retriever(search_kwargs={'k': 5}),
    return_source_documents=True
  )

  # we can now execute queries against our Q&A chain
  result = qa_chain({'query': query})
  return result

In [22]:
question = "What is the polution?"
ans = answer(question)
print(ans['result'])
print("----------")
print(ans['source_documents'])

 Pollution is the introduction by man into the environment of substances or energy liable to cause hazards to human health, harm to living resources and ecological systems, damage to structure or amenity or interference with legitimate uses of the environment.
----------
[Document(page_content='North Asian International Research Journal  of Social Science & Humanities   ISSN: 24 54-9827    Vol. 3, Issue 8, Aug.  2017  \n \nNorth Asian International research Journal consortiums www.nairjc.com  \n 152 DEFINITION OF POLLUTION  \n \nThe Royal Commission on Environmental Pollution in U.K. in its third report gave the following \ndefinition to the term “Pollution”, namely:  \n \nThe introduction by man into the environment of substances or energy liabl e to cause hazards to human \nhealth, harm to living resources and ecological systems, damage to structure or amenity or interference with \nlegitimate uses of the environment”.  \n \nAccording to Section 1(3) of the U.K. Environment Protectio